In [34]:
import numpy as np
import tensorflow as tf

In [11]:
def integration_task(seq_len, num_sapmles):
  noise = np.random.rand(seq_len) * 2 - 1
  target = np.sum(noise) >= 0
  for i in range(num_sapmles):
    yield noise, target

(array([ 0.13196764, -0.29037835, -0.8712971 , -0.82265928, -0.34207649]), False)
(array([ 0.13196764, -0.29037835, -0.8712971 , -0.82265928, -0.34207649]), False)
(array([ 0.13196764, -0.29037835, -0.8712971 , -0.82265928, -0.34207649]), False)
(array([ 0.13196764, -0.29037835, -0.8712971 , -0.82265928, -0.34207649]), False)
(array([ 0.13196764, -0.29037835, -0.8712971 , -0.82265928, -0.34207649]), False)
(array([ 0.13196764, -0.29037835, -0.8712971 , -0.82265928, -0.34207649]), False)
(array([ 0.13196764, -0.29037835, -0.8712971 , -0.82265928, -0.34207649]), False)


In [23]:
seq_len = 10
num_sapmles = 50

def my_integration_task():
  for i in integration_task(seq_len, num_sapmles):
    yield i

In [35]:
ds = tf.data.Dataset.from_generator(my_integration_task, output_signature=(
         tf.TensorSpec(shape=(seq_len,), dtype=tf.float64),
         tf.TensorSpec(shape=(), dtype=tf.int32)))



<FlatMapDataset shapes: ((10,), ()), types: (tf.float64, tf.int32)>


In [ ]:
class CustomSimpleRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, kernel_regularizer=None ):
        super(CustomSimpleRNNCell, self).__init__()
        
        self.units = units
        
        self.dense_forget = tf.keras.layers.Dense(units, 
                                                  activation=None,
                                                 kernel_regularizer=kernel_regularizer, 
                                                 use_bias=False)

        self.forget_bias = tf.Variable(tf.ones(units), name="RNN_forget_biases")

        self.dense_input = tf.keras.layers.Dense(units, 
                                                  activation=None,
                                                 kernel_regularizer=kernel_regularizer, 
                                                 use_bias=False)
        
        self.input_bias = tf.Variable(tf.zeros(units), name="RNN_input_biases")

        self.dense_hstate = tf.keras.layers.Dense(units, 
                                                  activation=None,
                                                  kernel_regularizer=kernel_regularizer, 
                                                  use_bias=False)
        
        self.hstate_bias = tf.Variable(tf.zeros(units), name="RNN_state_biases")

        self.dense_output = tf.keras.layers.Dense(units, 
                                                  activation=None,
                                                  kernel_regularizer=kernel_regularizer, 
                                                  use_bias=False)
        
        self.output_bias = tf.Variable(tf.zeros(units), name="RNN_output_bias")
        
        self.state_size = units
        
    @tf.function
    def call(self, input_t, state, training=False):
        
        # we compute the sum of the input at t matrix multiplied and the previous state matrix multiplied
        # and an additional bias added.
        
        f_t = tf.keras.activations.sigmoid(self.dense_forget(np.concatenate([state[0], input_t])) + self.forget_bias)

        i_t = tf.keras.activations.sigmoid(self.dense_input(np.concatenate([state[0], input_t])) + self.input_bias)

        C_t = tf.keras.activations.tanh(self.dense_hstate(np.concatenate([state[0], input_t])) + self.hstate_bias)

        o_t = tf-keras.activations.sigmoid(self.dense_output(np.concatenate([state[0], input_t])) self.output_bias)

        C_t = tf.math.multiply(f_t, state[1]) + tf.math.multiply(i_t, C_t)

        h_t = tf.math.multiply(o_t, tf.keras.activations.tanh(C_t))

        state = (h_t, C_t)
        
        return state

In [ ]:
class RNNLayer(tf.keras.layers.Layer):
  def __init__(self, cell):
    self.cell = cell

  @tf.function
  def call(self, x, states):
    i = tf.constant(0)
    c = lambda x2
    b = lambda x2, m: [i+1, tf.concat([m, m], axis=0)]
    tf.while_loop(
        c, b, loop_vars=[x],
        shape_invariants=[x.get_shape()])

In [ ]:
class RNNModel(tf.keras.layers.Layer):
    def __init__(self, RNN_Cell, return_sequences=False):
        super(RNNModel, self).__init__()
        
        self.return_sequences = return_sequences
        
        self.cell = RNN_Cell

    def call(self, data, training=False):
        
        length = data.shape[1]
        
        # initialize state of the simple rnn cell
        state = tf.zeros((data.shape[0], self.cell.units), tf.float32)
        
        # initialize array for hidden states (only relevant if self.return_sequences == True)
        hidden_states = tf.TensorArray(dtype=tf.float32, size=length)

        for t in tf.range(length):
            input_t = data[:,t,:]

            state = self.cell(input_t, state, training)

            if self.return_sequences:
                # write the states to the TensorArray
                #hidden_states = hidden_states.write(t, state)
                hidden_states.append(state)
        
        if self.return_sequences:
            # transpose the sequence of hidden_states from TensorArray accordingly 
            #(batch and time dimensions are otherwise switched after .stack())
            outputs = tf.transpose(hidden_states.stack(), [1,0,2])
        
        else:
            # take the last hidden state of the simple rnn cell
            outputs = state
        
        return outputs


In [ ]:
tf.keras.backend.clear_session()

#For showcasing we only use a subset of the training and test data (generally use all of the available data!)
#train_dataset = train_dataset.take(1000)
#test_dataset = test_dataset.take(100)

### Hyperparameters
num_epochs = 15
learning_rate = 0.0005

# Initialize the model.
model = RNNModel()
# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.BinaryCrossentropy()
# Initialize the optimizer: SGD with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.SGD(learning_rate)

# Initialize lists for later visualization.
train_losses = []

test_losses = []
test_accuracies = []

#testing once before we begin
test_loss, test_accuracy = test(model, test_ds, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, _ = test(model, train_ds, cross_entropy_loss)
train_losses.append(train_loss)

# We train for num_epochs epochs.
for epoch in range(num_epochs):
    print(f'Epoch: {str(epoch)} starting with accuracy {test_accuracies[-1]}')

    #training (and checking in with training)
    epoch_loss_agg = []
    for input,target in train_ds:
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        epoch_loss_agg.append(train_loss)
    
    #track training loss
    train_losses.append(tf.reduce_mean(epoch_loss_agg))

    #testing, so we can track accuracy and test loss
    test_loss, test_accuracy = test(model, test_ds, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)